# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():

    # All of these should at least capture the year
    pattern = (
        # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
        r'(?P<month>\d{1,2})[\/-](?P<day>\d{1,2})[\/-](?P<year>\d{2,4})|'
        # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
        # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
        # Feb 2009; Sep 2009; Oct 2010
        r'(?:(?P<day_alt>\d{1,2})\s)?'  # Optional day before the month
        r'(?P<month_alt>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*'  # Month (full/abbr.)
        r'(?:-|\.|\s|,)\s?'  # Separator handling
        r'(?P<day_alt2>\d{1,2})?[a-z]*'  # Optional day after month
        r'(?:-|,|\s)?\s?'  # More separators
        r'(?P<year_alt>(?:19|20)\d{2})|'  # Year (4-digit only)
        # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        r'(?P<day_alt3>\d{1,2})\s'  # Day first
        r'(?P<month_alt2>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*'  # Month
        r'[.,\s-]?\s?'  # Separator
        r'(?P<year_alt2>(?:19|20)\d{2})|'  # Year
        # 6/2008; 12/2009; 2009; 2010
        r'(?:(?P<month_slash>\d{1,2})[\/-])?(?P<year_only>(?:19|20)\d{2})'  # Month is optional, but year is required
    )
    # YOUR CODE HERE
    #raise NotImplementedError()

    # Extract date components
    order = df.astype(str).str.extract(pattern)

    # Fill missing values from alternative regex groups
    order['month'] = order['month'].fillna(order['month_alt']).fillna(order['month_slash'])
    order['day'] = order['day'].fillna(order['day_alt']).fillna(order['day_alt2'])
    order['year'] = order['year'].fillna(order['year_alt']).fillna(order['year_only'])

    # Convert month names to numbers
    month_map = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12',
        'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
        'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
    }
    order['month'] = order['month'].replace(month_map)

    # Convert two-digit years to four-digit (assuming 1900s for values < 25)
    order['year'] = order['year'].astype(str).apply(lambda x: f'19{x}' if len(x) == 2 else x)

    # Fill NaN values with empty strings
    order['month'] = order['month'].fillna('1')
    order['day'] = order['day'].fillna('1')
    
    # Convert to datetime for proper sorting
    order['date'] = pd.to_datetime(order[['year', 'month', 'day']].apply(lambda x: f"{x['year']}-{x['month']}-{x['day']}", axis=1))
    
    # Create a tuple of (date, index) for tie-breaking sort
    date_index_pairs = list(enumerate(order['date']))
    
    # Sort by date first, then by original index
    sorted_pairs = sorted(date_index_pairs, key=lambda x: (x[1], x[0]))
    
    # Return Series of sorted indices
    return pd.Series([pair[0] for pair in sorted_pairs])

In [3]:
order = date_sorter()

order

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64